In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [ ]:
df = pd.read_csv('app_streamlit/data/films_db.csv')

In [ ]:
# Utiliser MultiLabelBinarizer pour binariser les colonnes
mlb = MultiLabelBinarizer()
df = df.drop(['tconst','Titre_original','Nb_votes', "overview", "poster_path"], axis=1)

In [ ]:
df['Genres'] = df['Genres'].str.split(',')
df['liste_acteurs'] = df['liste_acteurs'].str.split(',')
df['Realisateur'] = df['Realisateur'].str.split(',')

In [ ]:
genres_binarized = pd.DataFrame(mlb.fit_transform(df['Genres']), columns=mlb.classes_)
acteurs_binarized = pd.DataFrame(mlb.fit_transform(df['liste_acteurs']), columns=mlb.classes_)
real_binarized = pd.DataFrame(mlb.fit_transform(df['Realisateur']), columns=mlb.classes_)

In [ ]:
df = pd.concat([df, genres_binarized, acteurs_binarized, real_binarized], axis=1)
df = df.drop(['Genres', 'liste_acteurs', 'Realisateur'], axis=1)

In [ ]:
scaler = MinMaxScaler()
df['Note_moyenne'] = scaler.fit_transform(df['Note_moyenne'].values.reshape(-1, 1))
df['popularity'] = scaler.fit_transform(df['popularity'].values.reshape(-1, 1))
df['Duree'] = scaler.fit_transform(df['Duree'].values.reshape(-1, 1))

In [ ]:
# normalisation des dates
# Conversion des dates en nombres de jours depuis le 1er janvier 1970
days_since_epoch = np.array([datetime.strptime(date, '%Y-%m-%d').date().toordinal() - datetime(1970, 1, 1).date().toordinal() for date in df['Annee']])

In [ ]:
# Normalisation des valeurs numériques
scaler = MinMaxScaler()
days_since_epoch_normalized = scaler.fit_transform(days_since_epoch.reshape(-1, 1))

In [ ]:
df = df.drop(['Annee'], axis=1)
# Ajout des valeurs normalisées au dataframe
df['Annee'] = days_since_epoch_normalized

In [ ]:
df.to_csv('from_predict.csv', index=False)

In [ ]:
# Application de la PCA
df = df.drop(['Titre'], axis=1)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(df)
X =pd.DataFrame(X_pca)
# on transform le numpy en dataframe
X.to_csv('from_predict_PCA.csv', index=False)

In [ ]:
genres_recherches = ["Action", "Science-Fiction"]
masque = films_db["Titre"].isin(films_db[films_db["Genres"].str.split(",").explode().isin(genres_recherches)]["Titre"])
films_recherches = films_db[masque]